In [18]:
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, Label, Layout
import ipywidgets as widgets
import math

In [ ]:
# Present values go here

In [51]:
def cavity_radius(kt):
    if kt < 1:
        return 0
    else:
        return 17 * math.pow(math.log10(kt), 1/3)

def cavity_volume(kt):
    return (4/3) * math.pi * math.pow(kt, 3)

def depth_estimate(kt):
    if kt < 1:
        return 0
    else:
        return 120*math.pow(math.log10(kt), 1/3)
    
def spacing_estimate(kt):
    return 2*kt + 30

def yield_estimator(mb, depth=0, propagation=4.25):
    if(mb==0):
        print("Yield: 0 kilotons")
        return
    if(mb < 4):
        if(depth==0):
            yld = math.pow(10, (mb-propagation))
        else:
            # I don't entirely trust the calculations here, tbh. Less important for small explosions atm, todo.
            yld = math.pow(10, (
                mb-(propagation+1.63736)+(0.7875*math.log10(depth))/1.35
                           ))
    else:
        if(depth==0):
            yld = math.pow(10, (mb-propagation)/0.75)
        else:
            yld = math.pow(10, (
                mb-(propagation+1.63736)+(0.7875*math.log10(depth))/1.0125
                           ))
    
    print("Yield: ", "{0:.3f}".format(yld), " kilotons")
    print("Cavity Radius: ", "{0:.0f}".format(cavity_radius(yld)), " meters")
    print("Cavity Volume: ", "{0:.0f}".format(cavity_volume(yld)), " cubic meters")
    print("Minimum burial depth: ", "{0:.0f}".format(depth_estimate(yld)), " meters (VERY rough estimate)")
    print("Minimum spacing for equivalent test: ", "{0:.0f}".format(spacing_estimate(yld)), 
          " meters (VERY rough estimate)")

In [52]:
mb_input = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=50,
    step=0.1,
    description='Mb: ',
    disabled=False
)
depth_input = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100000,
    step=1,
    description='Depth (m): ',
    disabled=False
)
prop_text = widgets.BoundedFloatText(
    value=4.25,
    min=3,
    max=5,
    step=0.01,
    description='Propagation: ',
    disabled=False
)
prop_slider = widgets.FloatSlider(
    value=4.25,
    min=3,
    max=5,
    step=0.01,
    description='',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)
# set layout
ui_layout = Layout(
    grid_row='4',
    grid_column='1'
)

# Nuclear Test Yield Calculator

Input the magnitude (Mb), as well as depth and propagation if available.

(todo) Alternatively, choose a preset to see historic nuclear estimates.

In [53]:
# link any elements
prop_link = widgets.jslink((prop_text, 'value'), (prop_slider, 'value'))

# build the final UI
ui = widgets.GridBox([mb_input, depth_input, prop_text, prop_slider],
                    layout=ui_layout)

# this is the important run stuff bit
yield_estimator_interactive = widgets.interactive_output(
    yield_estimator, {'mb': mb_input, 'depth': depth_input, 'propagation': prop_text})

# all done
display(ui, yield_estimator_interactive)

GridBox(children=(BoundedFloatText(value=0.0, description='Mb: ', max=50.0, step=0.1), BoundedIntText(value=0,…

Output()

# Sources & Thanks

This site was built and maintained by [@madwonk](https://twitter.com/madwonk) to streamline work done by [Jeffrey Lewis](https://twitter.com/ArmsControlWonk) and bring it to a larger audience.

## How does this work?
This site is a Jupyter notebook exported with [Voilà](https://github.com/voila-dashboards/voila). The source code is available on [Github](https://github.com/BenMueller)!

## I've found a bug!
Feel free to file a report on Github, or send me a message @madwonk on Twitter.

## Sources
[Punggye-ri yield, no depth correction](http://dx.doi.org/10.1785/0120100202)

[Punggye-ri with depth correction](https://doi.org/10.1002/grl.50607)

[Novaya Zemlya](www.jstor.org/stable/26977)

[Pokhran (high estimate from BARC)](http://web.archive.org/web/20010720043355/http://www.barc.ernet.in/webpages/milestones/drs_03.html)

[Moruroa](https://apps.dtic.mil/dtic/tr/fulltext/u2/a228258.pdf)

[Chagai (corrected for tectonic activity)](https://apps.dtic.mil/dtic/tr/fulltext/u2/a228258.pdf)
